In [1]:
# print version / should be 1.6.3
println(versioninfo())

# load centralized paths dictionary
using YAML
PATHS_FILE = "../../../PATHS.yml"
PATHS = YAML.load_file(PATHS_FILE);

"""
    load_path(keys::Vector{String}):String

Return the absolute path for a requested item within the nested PATHS dictionary.
"""
function loadpath(keys::Vector{String}):String
    # recusively assemble paths from keys
    requested_path = foldl((x, y) -> getindex(x, y), keys; init=PATHS)
    # rephrase abs path from relative paths to be platform independent.
    realpath(joinpath(splitdir(realpath(PATHS_FILE))[1], requested_path))
end;

Julia Version 1.6.3
Commit ae8452a9e0 (2021-09-23 17:34 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU E5-2630 v4 @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, broadwell)
Environment:
  __LMOD_REF_COUNT_JULIA_DEPOT_PATH = /home/biotoml/.julia:1;/sw/comp/julia/1.6.3/rackham/lib/glob_pkg:1
  JULIA_DEPOT_PATH = /home/biotoml/lsm/src/.julia/v1.6_depot:/home/biotoml/.julia:/sw/comp/julia/1.6.3/rackham/lib/glob_pkg
  __LMOD_REF_COUNT_JULIA_LOAD_PATH = @:1;@v#.#:1;@stdlib:1;/sw/comp/julia/1.6.3/rackham/lib/glob_pkg/environments/v1.6:1
  JULIA_PROJECT = /home/biotoml/lsm/src/.julia/v1.6_regrid
  JULIA_ROOT = /sw/comp/julia/1.6.3/rackham
  JULIA_LOAD_PATH = @:@v#.#:@stdlib:/sw/comp/julia/1.6.3/rackham/lib/glob_pkg/environments/v1.6
nothing


In [317]:
using NaNMath; nm=NaNMath
using NCDatasets
using Revise
using StatsBase

In [11]:
file = "/home/biotoml/lsm/src/.julia/v1.7_depot/artifacts/f3a9530cb23714c5d2a3863a7317ff2bfb32077a/BIOMASS_SHOOT_120X_1Y_V2.nc"

"/home/biotoml/lsm/src/.julia/v1.7_depot/artifacts/f3a9530cb23714c5d2a3863a7317ff2bfb32077a/BIOMASS_SHOOT_120X_1Y_V2.nc"

In [12]:
ds = NCDataset(file, "r");

In [13]:
typeof(ds[:data])

NCDatasets.CFVariable{Float64, 2, NCDatasets.Variable{Float64, 2, NCDataset{Nothing}}, NCDatasets.Attributes{NCDataset{Nothing}}, NamedTuple{(:fillvalue, :scale_factor, :add_offset, :calendar, :time_origin, :time_factor), NTuple{6, Nothing}}}

In [14]:
describe(ds["lon"]), describe(ds["lat"])

Summary Stats:
Length:         43200
Missing Count:  0
Mean:           -0.000000
Minimum:        -179.995833
1st Quartile:   -89.997917
Median:         0.000000
3rd Quartile:   89.997917
Maximum:        179.995833
Type:           Float64
Summary Stats:
Length:         21600
Missing Count:  0
Mean:           0.000000
Minimum:        -89.995833
1st Quartile:   -44.997917
Median:         0.000000
3rd Quartile:   44.997917
Maximum:        89.995833
Type:           Float64


(nothing, nothing)

In [15]:
include("./regridding.jl")

Regridder2D

In [152]:
rgo = Regridder2D(180,360)

Regridder2D{Int64, Float64}(RegridAxis{Int64, Float64}[RegridAxis{Int64, Float64}("lat", -90.0, 90.0, 180), RegridAxis{Int64, Float64}("lon", -180.0, 180.0, 360)])

In [38]:
function init_regridded(regridder::RG; ft::Type{N}=Float64) where {N<:Number, RG<:Regridder}
    zeros(ft, size(regridder))
end

function regrid_dataset(regridder::RG, ncds::NCDataset) where RG<:Regridder
    regridded_ncds = init_regridded(regridder)
end

regrid_dataset (generic function with 1 method)

In [39]:
using IntervalSets

In [40]:
function RegridAxis(name::String, pointrange::AbstractArray{FT}) where FT<:AbstractFloat
    prsz½ = stepsize(pointrange) / 2
    prmin = minimum(pointrange) - prsz½
    prmax = maximum(pointrange) + prsz½
    prcard = length(pointrange)
    
    RegridAxis{Int64, FT}(name, prmin, prmax, prcard)
end

function minmaxdist(pointrange::AbstractArray{FT}) where FT<:AbstractFloat
    diff([extrema(pointrange)...])[1] / length(pointrange) * (length(pointrange)+1)
end

stepsize(pointrange::AbstractArray{FT}) where FT<:AbstractFloat = minmaxdist(pointrange)/length(pointrange)


function intsect_intervals(src_itvs::AT, dest_itv::Interval) where {AT<:AbstractArray}
    # first get width and index for intersecting intervals
    map(
        ((j, itv),) -> 
        let isct = dest_itv ∩ itv
                isempty(isct) ? nothing : [j, width(isct)/width(itv)]
        end, 
        enumerate(src_itvs)) |>
    # filter the nothings
    x -> filter(!isnothing, x) |>
    # adjust the type of the array
    x -> isempty(x) ? (return nothing) : convert(Array{eltype(x).b}, x) |>
    x -> reduce(hcat, x)
end

intsect_intervals (generic function with 1 method)

In [318]:
rgao = RegridAxis("aa", -180., 180., 360)
rgao2 = RegridAxis("exp", ds["lon"][:])

RegridAxis{Int64, Float64}("exp", -179.99999999999778, 179.99999999999778, 43200)

In [320]:
a = intsect_intervals(intervals(rgao2), intervals(rgao)[1])

ivs = map(ax -> intervals(ax)[100], [rgo.axes[2],rgo.axes[1]])

120-element Vector{Float64}:
   1.0
   2.0
   3.0
   4.0
   5.0
   6.0
   7.0
   8.0
   9.0
  10.0
  11.0
  12.0
  13.0
   ⋮
 109.0
 110.0
 111.0
 112.0
 113.0
 114.0
 115.0
 116.0
 117.0
 118.0
 119.0
 120.0

In [326]:
function extract_cell(ncds::NCDataset, itvs::AbstractArray, variables::AbstractArray{String})
    ncdims = keys(ncds.dim)
    ncdim_vects = map(var -> intervals(RegridAxis(var, ncds[var][:])), ncdims)
    map(var -> extract_subarray(ncds[var], ncdim_vects, itvs), variables)
end

function extract_subarray(data::NCDatasets.CFVariable, dimension_vectors::AbstractArray, dest_itvs::AbstractArray)
    index_weights = map(((dim_itvs, dest_itv),) -> intsect_intervals(dim_itvs, dest_itv), zip(dimension_vectors, dest_itvs))
    indices = map(j -> convert(Array{Int}, index_weights[j][1,:]), 1:length(index_weights))
    weigths =  map(j -> index_weights[j][2,:], 1:length(index_weights))

    data[indices...], ortho_multiply(weigths')
end

# simple extension to the 
nm_weighted(data, weights, operation) = operation(data.*weights)

function regrid_cell(ncds::NCDataset, itvs::AbstractArray, variables::AbstractArray{String}, operation::Function)
    map(((dt,wt),) -> nm_weighted(dt, wt, operation), extract_cell(ncds, itvs, variables))
end

function ortho_multiply(vec1::AbstractArray, vec2::AbstractArray)
    let z=vec1'*vec2
        reshape(z, (1, length(z)))
    end
end

function ortho_product(ortho_bases::AbstractArray)
    flat_tensor = reduce(ortho_multiply, ortho_bases)
    reshape(flat_tensor, (length.(ortho_bases)...))
end

regrid_cell (generic function with 1 method)

In [328]:
j = regrid_cell(ds, ivs, ["data", "std"], nm.mean)

2-element Vector{Float64}:
   0.11253981541991022
 NaN

In [165]:
wsum(j[1])

LoadError: MethodError: no method matching wsum(::Matrix{Float64})
[0mClosest candidates are:
[0m  wsum(::AbstractArray, [91m::AbstractVector{T} where T[39m, [91m::Colon[39m) at /home/biotoml/.julia/packages/StatsBase/548SN/src/weights.jl:357
[0m  wsum(::AbstractArray{var"#s27", N} where {var"#s27"<:Number, N}, [91m::UnitWeights[39m, [91m::Int64[39m) at /home/biotoml/.julia/packages/StatsBase/548SN/src/weights.jl:553
[0m  wsum(::AbstractArray{T, N} where N, [91m::AbstractVector{W}[39m, [91m::Int64[39m) where {T<:Number, W<:Real} at /home/biotoml/.julia/packages/StatsBase/548SN/src/weights.jl:548
[0m  ...

In [215]:
x =[1 2]
y = [3 4]

1×2 Matrix{Int64}:
 3  4

In [216]:
vectors = [x, x, y]

2×2 Matrix{Int64}:
 3  4
 6  8

ortho_product (generic function with 1 method)